In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

### Загрузка данных

In [2]:
data = pd.read_excel('data/data_ford_price.xlsx') 

#### Предобработка данных

In [3]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

### Метод рекурсивного исключения признаков RFE

In [4]:
from sklearn.feature_selection import RFE

#### Отбор признаков по методу рекурсивного исключения признаков RFE

In [5]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
print('Признаки, отобранные RFE:',  selector.get_feature_names_out())

X_train_RFE = X_train[selector.get_feature_names_out()]
X_test_RFE = X_test[selector.get_feature_names_out()]

Признаки, отобранные RFE: ['year' 'cylinders' 'lat']


#### Обучение модели

In [6]:
model = LinearRegression()
model.fit(X_train_RFE, y_train)
y_predicted_RFE = model.predict(X_test_RFE)
mae_RFE = mean_absolute_error(y_test, y_predicted_RFE)
print('MAE RFE: %.3f' % mae_RFE)

MAE RFE: 5096.570


### Метод выбора k лучших переменных SelectKBest

In [7]:
from sklearn.feature_selection import SelectKBest, f_regression

#### Отбор признаков по методу SelectKBest

In [8]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
print('Признаки, отобранные SelectKBest:',  selector.get_feature_names_out())

X_train_KBest = X_train[selector.get_feature_names_out()]
X_test_KBest = X_test[selector.get_feature_names_out()]

Признаки, отобранные SelectKBest: ['year' 'cylinders' 'odometer']


#### Обучение модели

In [9]:
model = LinearRegression()
model.fit(X_train_KBest, y_train)
y_predicted_KBest = model.predict(X_test_KBest)
mae_KBest = mean_absolute_error(y_test, y_predicted_KBest)
print('MAE KBest: %.3f' % mae_KBest)

MAE KBest: 4708.946


### Сравнение двух методов отбора признаков

In [10]:
print('Сравнение MAE двух методов отбора признаков')
print('MAE RFE: %.3f' % mae_RFE)
print('MAE KBest: %.3f' % mae_KBest)

Сравнение MAE двух методов отбора признаков
MAE RFE: 5096.570
MAE KBest: 4708.946


#### Вывод:
>На тестовой выборке наилучший результат показал метод отбора признаков SelectKBest